### Wybór i zdefiniowanie metody klasyfikacji danych
#### Do zakodowania etykiet wykorzystałem bibliotekę sklearn i LabelEncoder, który koduje etykiety klasowe (w moim przypadku poszczególne języki) na liczby całkowite
#### Do transformacji tekstu wykorzystałem sentence transformer "all-MiniLM-L6-v2" ze względu na jego niewilki rozmiar i niską złożoność obliczeniową, a co za tym idzie dobrą wydajność.

In [2]:
from typing import List
from sklearn.preprocessing import LabelEncoder 
import sentence_transformers
from sentence_transformers import SentenceTransformer
label_encoder = LabelEncoder() # pobranie narzędzia do kodowania etykiet klasowych dla rodzajów języków
sentence_transformer = SentenceTransformer("all-MiniLM-L6-v2") # wykorzystanie sentence transformera "all-MiniLM-L6-v2"

/home/student/ML_Project/venv_ml_project/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/student/ML_Project/venv_ml_project/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Wczytanie wcześniej przygotowanych danych oraz ich transformacja, sprawdzenie i zapisanie do pliku

In [3]:
# Tworzę funkcję, które za argumenty bierze model oraz liste tekstow i zwraca ich embeddingi
def get_embeddings(model, texts: List[str]):
    return model.encode(texts)

In [4]:
# Wczytuje mój przygotowany df w całości i sprawdzam czy ma długość 70000
import pandas as pd 
df = pd.read_csv("dataset/languages_prep.csv")
assert len(df) == 70000

In [5]:
# Zapisuje texty i labele do listy 
labels = df.labels.tolist()
texts = df.text.tolist()

In [6]:
# Koduję etykiety za pomocą label enkodera
enc_labels = label_encoder.fit_transform(labels)

### Tworzę embeddingi dla każdego textu w texts - długi czas trwania w moim przypadku około 2-3h

In [7]:
# Tworzę embeddingi dla każdego textu w texts - długi czas trwania w moim przypadku około 2-3h
embeddings = get_embeddings(sentence_transformer, texts)

In [8]:
# Sprawdzam ile wymiarów ma wektor o indeksie 0
len(embeddings[0])

384

In [9]:
# Sprawdzam jaką ma długość
len(embeddings)

50000

In [10]:
# Sprawdzam kształt embendingsów
embeddings.shape

(50000, 384)

In [11]:
len(enc_labels)

50000

In [12]:
# Przypisuję enc_labels i embeddingi do kolumn w moim df
df = df.assign(enc_labels = [l for l in enc_labels])
df = df.assign(embeddings = [e for e in embeddings])
df.head(n=10)

,Unnamed: 0,labels,text,enc_labels,embeddings
0,0,pt,"os chefes de defesa da estónia, letónia, lituâ...",12,"[0.036873266, 0.0020223923, -0.09351195, 0.001..."
1,1,bg,размерът на хоризонталната мрежа може да бъде ...,1,"[0.019863939, 0.06454754, -0.041304275, -0.031..."
2,2,zh,很好，以前从不去评价，不知道浪费了多少积分，现在知道积分可以换钱，就要好好评价了，后来我就把...,19,"[-0.041812006, 0.0039601177, 0.01682547, -0.00..."
3,3,th,สำหรับ ของเก่า ที่ จริงจัง ลอง honeychurch ...,15,"[0.075852364, 0.053805254, 0.07748131, -0.0450..."
4,4,ru,Он увеличил давление .000000000000000000000000...,13,"[-0.04810861, 0.03578605, 0.011757057, 0.02441..."
5,5,pl,"S Jak sobie życzysz: Widzisz, jak Hitler zabij...",11,"[-0.101992615, 0.13540268, -0.109019354, 0.036..."
6,6,ur,اس کے بارے میں ، سفید شادی کی شرح کے بعد سفید ...,17,"[0.03445083, 0.07025945, -0.013857332, -0.0287..."
7,7,sw,Zabuni ya ushindani pia imekuwa rahisi kwa sif...,14,"[-0.027691623, 0.10228121, 0.030589335, -0.032..."
8,8,tr,Devasa 12 yüzyıl abbatiale saint-Pierre-Et-Sai...,16,"[-0.04013098, 0.09787849, -0.06950884, -0.0368..."
9,9,ur,موجودہ اثاثوں میں سے ایک کا اضافہ ہو سکتا ہے ۔...,17,"[0.07042703, 0.06137885, -0.016436763, 0.00872..."


In [13]:
# Zapisuję df jako feather format train_with_embeddings.bin 
df.to_feather("dataset/languages_with_embeddings.bin")